# This notebook is meant to showcase computing attention with a causal mask

# Call Imports and Seed the Random Number Generators

In [18]:
import torch
import numpy as np
from torch import nn

# Set this to whatever you want
seed = 10

torch.manual_seed(seed)
np.random.seed(seed)

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Make Random Q, K, and V tensors

usually the inputs to the attention function get projected but for simplicity we'll just assume it happened already

`T` for timestep  
`d` for dimensionality

In [2]:
T, d = 3, 4
q, k, v = torch.rand((T, d*3)).chunk(3, dim=-1)

print(f'q shape: {q.shape}')
print(f'k shape: {k.shape}')
print(f'v shape: {v.shape}')

q shape: torch.Size([3, 4])
k shape: torch.Size([3, 4])
v shape: torch.Size([3, 4])


# Time to Compute Attention

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^{\top}}{\sqrt{d}}\right)V
$$

# First Compute $\frac{QK^{\top}}{\sqrt{d}}$

This operation outputs a (T, T) matrix  
The $n$th row refers to the $n$th token in the sequence  
The $n$th column refers to the "attention score"

In [3]:
qk = (q@k.T)/np.sqrt(d)

print(qk)

tensor([[1.0366, 0.4480, 0.8684],
        [0.6112, 0.2692, 0.4616],
        [0.3284, 0.1526, 0.4017]])


# Masking is applied by setting masked values to $-\infty$ before softmax
$\oslash$ means element-wise division
$$
\text{softmax}(x) = \exp(x) \oslash \sum\limits_{i}\exp(x_i)
$$

In [7]:
# random example vector
example = torch.rand(5)
print(f'v: {example}')

# Set last value to -inf
example[-1] = float('-inf')
print(f'\nv with masked end value: {example}')

# apply softmax after setting last value to -inf
s_example = torch.softmax(example, dim=0)
print(f'\napply softmax: {s_example}')

v: tensor([0.1602, 0.6989, 0.0781, 0.5008, 0.2713])

v with masked end value: tensor([0.1602, 0.6989, 0.0781, 0.5008,   -inf])

apply softmax: tensor([0.1984, 0.3400, 0.1827, 0.2789, 0.0000])


# Now Apply Causal Attention Mask and Compute Softmax

In [8]:
# Create attention mask
attn_mask = torch.triu(
    torch.ones((T, T), dtype=torch.bool),
    diagonal=1
)
print('Attention mask (True values are the ones that get masked out):')
print(attn_mask)

# Apply attention mask
sqk = qk.clone()
sqk[attn_mask] = float('-inf')
sqk = torch.softmax(sqk, dim=-1)
print('\nQK^T/(sqrt(d)) after applying softmax:')
print(sqk)


Attention mask (True values are the ones that get masked out):
tensor([[False,  True,  True],
        [False, False,  True],
        [False, False, False]])

QK^T/(sqrt(d)) after applying softmax:
tensor([[1.0000, 0.0000, 0.0000],
        [0.5847, 0.4153, 0.0000],
        [0.3431, 0.2878, 0.3692]])


# Now Matrix Multiply with V and you have your attention output

In [9]:
attn_out = sqk@v

print('output:')
print(attn_out)

output:
tensor([[0.2759, 0.8454, 0.4397, 0.1585],
        [0.4682, 0.7469, 0.2908, 0.4935],
        [0.6005, 0.5586, 0.2971, 0.6820]])


# Compare with Original V

In [10]:
print('V:')
print(v)

print('\nQK^T/(sqrt(d)) after applying softmax:')
print(sqk)

print('\nAttention Output:')
print(attn_out)

V:
tensor([[0.2759, 0.8454, 0.4397, 0.1585],
        [0.7389, 0.6082, 0.0813, 0.9651],
        [0.7942, 0.2534, 0.3329, 0.9478]])

QK^T/(sqrt(d)) after applying softmax:
tensor([[1.0000, 0.0000, 0.0000],
        [0.5847, 0.4153, 0.0000],
        [0.3431, 0.2878, 0.3692]])

Attention Output:
tensor([[0.2759, 0.8454, 0.4397, 0.1585],
        [0.4682, 0.7469, 0.2908, 0.4935],
        [0.6005, 0.5586, 0.2971, 0.6820]])


# Notice that row 0 of attention output is just the first value vector

In [11]:
attn_out[0] == v[0]

tensor([True, True, True, True])

# Now see that row 1 of attention output is just a linear combination of the first 2 rows of V

In [12]:
# compare extracted output with manual computation
(sqk@v)[1] == sqk[1, 0]*v[0] + sqk[1, 1]*v[1]

tensor([True, True, True, True])

# Now see that row 2 of attention output is just a linear combination of all 3 rows of V

In [14]:
(sqk@v)[2] == sqk[2, 0]*v[0] + sqk[2, 1]*v[1] + sqk[2, 2]*v[2]

tensor([True, True, True, True])

# Test Pytorch Attention Module

In [37]:
d = 10
mod = nn.MultiheadAttention(d, num_heads=1, batch_first=True)
q, k, v = torch.randn(2, 3, 10*3).chunk(3, dim=-1)

pad_mask = torch.triu(torch.ones(v.shape[:2], dtype=torch.bool), diagonal=1)
out, out_weights = mod(q, k, v, key_padding_mask=pad_mask)

print('V:')
print(v, end='\n\n')

print('output:')
print(out, end='\n\n')

print('attention_weights')
print(out_weights)


V:
tensor([[[ 1.3097,  0.2201, -0.2880, -0.8347, -1.2838,  1.1809,  0.0583,
           2.3550, -0.0433,  0.6667],
         [ 0.1308, -0.8500, -0.1962, -0.5518, -0.6516,  0.4247, -1.4717,
          -0.1648, -1.2407,  0.2646],
         [ 1.4802, -1.3781,  0.4998,  0.2795, -1.2944, -0.3256, -1.6201,
           0.8233,  0.2742,  0.3658]],

        [[ 1.1157,  0.9831, -1.6072,  1.6858, -0.5771, -0.1047,  0.2066,
           1.0235, -0.4788, -2.2119],
         [ 0.0888,  0.4549, -1.1206,  0.9442,  0.1810, -0.6077,  0.1736,
           0.5794,  0.2213,  0.4951],
         [ 1.5629,  0.1023,  1.7757, -0.5724, -0.2848, -1.9658, -1.2562,
           0.4375,  0.5342,  2.2882]]])

output:
tensor([[[ 1.2394,  0.4094, -0.7183,  1.0527,  0.4229, -0.7041, -0.6246,
           0.5600,  0.3418, -0.0489],
         [ 1.2394,  0.4094, -0.7183,  1.0527,  0.4229, -0.7041, -0.6246,
           0.5600,  0.3418, -0.0489],
         [ 1.2394,  0.4094, -0.7183,  1.0527,  0.4229, -0.7041, -0.6246,
           0.5600,  0.3